In [3]:
# Notebook-Zelle 1: Imports & Graph laden
from rdflib import Graph, Namespace, RDF, URIRef
from pathlib import Path

# Namespace definieren
AG = Namespace("http://www.semanticweb.org/AgentProgramParams/")

# Pfad zur TTL-Datei (angepasst nach deinem Speicherort)
ttl_path = Path(r"D:\MA_Python_Agent\MSRGuard\KGs\ParamDiag_Agent.ttl")

# Graph laden
g = Graph()
g.parse(ttl_path.as_posix(), format="turtle")
print("Initiale Tripelanzahl:", len(g))



Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/1999/02/22-rdf-syntax-ns#XMLLiteral, Converter=<function _parseXML at 0x000001DBAF7DCB80>
Traceback (most recent call last):
  File "d:\MA_Python_Agent\.venv311\Lib\site-packages\rdflib\term.py", line 2163, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^
  File "d:\MA_Python_Agent\.venv311\Lib\site-packages\rdflib\term.py", line 1680, in _parseXML
    retval = xml.dom.minidom.parseString(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Alexander Verkhov\AppData\Local\Programs\Python\Python312\Lib\xml\dom\minidom.py", line 2000, in parseString
    return expatbuilder.parseString(string)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Alexander Verkhov\AppData\Local\Programs\Python\Python312\Lib\xml\dom\expatbuilder.py", line 922, in parseString
    return builder.parseString(string)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File

Initiale Tripelanzahl: 2505


In [4]:
# Notebook-Zelle 2: Löschroutine definieren
def delete_instances_of_class(graph, cls):
    """
    Löscht alle Tripel, bei denen eine Ressource den Typ `cls` hat,
    sowie alle Tripel, bei denen diese Ressource als Subjekt auftritt.
    """
    # finde alle Instanzen der Klasse
    instances = list(graph.subjects(RDF.type, cls))
    print(f"Gefundene Instanzen von {cls}: {len(instances)}")
    # für jede Instanz: alle Tripel mit dieser Instanz als Subjekt entfernen
    for inst in instances:
        graph.remove((inst, None, None))
    # ggf. auch Tripel, bei denen diese Instanz als Objekt verwendet wird, entfernen:
    for inst in instances:
        graph.remove((None, None, inst))
    return len(instances)

from rdflib.namespace import OWL

def delete_all_named_individuals(graph):
    """
    Löscht alle Tripel, in denen eine Ressource den Typ owl:NamedIndividual hat.
    Es werden sowohl Tripel entfernt, in denen die Ressource Subjekt ist,
    als auch Tripel, in denen sie als Objekt verwendet wird.
    """
    individuals = list(graph.subjects(RDF.type, OWL.NamedIndividual))
    print(f"Gefundene owl:NamedIndividual-Instanzen: {len(individuals)}")

    for inst in individuals:
        # alle Tripel mit inst als Subjekt entfernen
        graph.remove((inst, None, None))
        # alle Tripel mit inst als Objekt entfernen
        graph.remove((None, None, inst))

    # zusätzlich den Typ-Triple selbst entfernen (inst a owl:NamedIndividual)
    for inst in individuals:
        graph.remove((inst, RDF.type, OWL.NamedIndividual))

    return len(individuals)

# Aufruf:

# Notebook-Zelle 3: Instanzen löschen
# Lösche alle Instanzen der Klasse class_Program
num_prog = delete_instances_of_class(g, AG.class_Program)
print(f"Instanzen class_Program gelöscht: {num_prog}")
print("Tripel nach Löschen class_Program:", len(g))

# Lösche alle Instanzen der Klasse class_Variable
num_var = delete_instances_of_class(g, AG.class_Variable)
print(f"Instanzen class_Variable gelöscht: {num_var}")
print("Tripel nach Löschen class_Variable:", len(g))

num_named_ind = delete_all_named_individuals(g)
print(f"Gelöschte owl:NamedIndividual-Instanzen: {num_named_ind}")
print("Tripel nach Löschen owl:NamedIndividual:", len(g))

outpath = Path(r"D:\MA_Python_Agent\MSRGuard\KGs\ParamDiag_Agent_cleaned.ttl")
g.serialize(destination=outpath.as_posix(), format="turtle")
print("Bereinigter Graph unter:", outpath)
print("Finale Tripelanzahl:", len(g))




Gefundene Instanzen von http://www.semanticweb.org/AgentProgramParams/class_Program: 50
Instanzen class_Program gelöscht: 50
Tripel nach Löschen class_Program: 1784
Gefundene Instanzen von http://www.semanticweb.org/AgentProgramParams/class_Variable: 390
Instanzen class_Variable gelöscht: 390
Tripel nach Löschen class_Variable: 386
Gefundene owl:NamedIndividual-Instanzen: 218
Gelöschte owl:NamedIndividual-Instanzen: 218
Tripel nach Löschen owl:NamedIndividual: 86
Bereinigter Graph unter: D:\MA_Python_Agent\MSRGuard\KGs\ParamDiag_Agent_cleaned.ttl
Finale Tripelanzahl: 86
